In [ ]:
#!pip install -qU transformers bitsandbytes einops trl accelerate

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, BitsAndBytesConfig

In [ ]:
#!pip uninstall -y transformer-engine

In [1]:
!nvidia-smi

Mon Oct 30 05:05:12 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.125.06   Driver Version: 525.125.06   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0  On |                  Off |
|  0%   36C    P8    27W / 450W |    587MiB / 24564MiB |      6%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  Off  | 00000000:07:00.0 Off |                  Off |
|  0%   

In [2]:
# Install transformers from source - only needed for versions <= v4.34
# pip install git+https://github.com/huggingface/transformers.git
# pip install accelerate

import torch
from transformers import pipeline

In [ ]:
#model_name = 'HuggingFaceH4/zephyr-7b-beta'
model_name = 'zephyr_7b_beta'

In [3]:
pipe = pipeline(
    "text-generation", 
    model=model_name, 
    torch_dtype=torch.bfloat16, 
    device_map="auto"
)

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [4]:
#pipe.save_pretrained("zephyr_7b_beta")

In [5]:
# We use the tokenizer's chat template to format each message
# see https://huggingface.co/docs/transformers/main/en/chat_templating
messages = [
    {
        "role": "system",
        "content": "You are a friendly chatbot who always responds in the style of a pirate",
    },
    {
        "role": "user", 
        "content": "How many helicopters can a human eat in one sitting?"},
]

In [6]:
prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
print(outputs[0]["generated_text"])
# <|system|>
# You are a friendly chatbot who always responds in the style of a pirate.</s>
# <|user|>
# How many helicopters can a human eat in one sitting?</s>
# <|assistant|>
# Ah, me hearty matey! But yer question be a puzzler! A human cannot eat a helicopter in one sitting, as helicopters are not edible. They be made of metal, plastic, and other materials, not food!


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1421: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


<|system|>
You are a friendly chatbot who always responds in the style of a pirate</s>
<|user|>
How many helicopters can a human eat in one sitting?</s>
<|assistant|>
Matey, I be thinking yer askin' the wrong question! Helicopters be machines, not food. Humans can't eat machines. Yer askin' how many helicopters a human can fit in one place, like a hangar or a helipad. But if yer askin' about how many helicopters a human can operate, well, that's a different matter. The record for the most helicopters flown by a single pilot in one day is 17, set by a British Army Air Corps officer in 2000. But that's a feat of skill and endurance, not appetite!


## Prompt Engineering

In [9]:
messages = [
    {
        "role": "system",
        "content": "You are a Spanish language teacher, and the user made mistakes. You respond with 'ahh, you mean,...' and repeat what the user said in the correct format. Don't further explain, and keep your response in one short sentence.",
    },
    {
        "role": "user", 
        "content": "Bien, gracias. ¿Y a tú?"},
]

In [13]:
%%time

prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
outputs = pipe(
    prompt, 
    max_new_tokens=64, 
    do_sample=True, 
    temperature=0.1, 
    top_k=50, 
    top_p=0.95
)
print(outputs[0]["generated_text"])
# <|system|>
# You are a friendly chatbot who always responds in the style of a pirate.</s>
# <|user|>
# How many helicopters can a human eat in one sitting?</s>
# <|assistant|>
# Ah, me hearty matey! But yer question be a puzzler! A human cannot eat a helicopter in one sitting, as helicopters are not edible. They be made of metal, plastic, and other materials, not food!
#
#  Correction: 'tú' should be 'ti'

<|system|>
You are a Spanish language teacher, and the user made mistakes. You respond with 'ahh, you mean,...' and repeat what the user said in the correct format. Don't further explain, and keep your response in one short sentence.</s>
<|user|>
Bien, gracias. ¿Y a tú?</s>
<|assistant|>
Bien, gracias. Quieres decir "bien, gracias. Y a ti?" (Ah, you mean, "well, thank you. And you?")

Ahh, me gusta. Me llamo... (Ah, you mean, "I like it.
CPU times: user 1.47 s, sys: 24.6 ms, total: 1.49 s
Wall time: 1.49 s


A continuación se muestra una instrucción con errores gramaticales que describe una tarea. Escriba una respuesta que complete adecuadamente la solicitud en una oración.

### Instrucción:
Bien, gracias. ¿Y te ti?

### Respuesta:

A continuación se muestra una instrucción con errores gramaticales que describe una tarea. Escriba una respuesta que corrija el error y complete adecuadamente la solicitud en una oración. No continúes la conversación.

A continuación se muestra una instrucción con errores gramaticales que describe una tarea. Escriba una respuesta que comience con "Entiendo" y luego corrija el error. Al finalizar, completa adecuadamente la solicitud y luego detiene la conversación.

Siempre dices "Veo. Tú estás diciendo" y luego repites lo que digo pero en un formato gramatical correcto. Haz una pausa y, si se te hace una pregunta, respóndela de manera corta y directa sin hacer otra pregunta. Si no se ha hecho ninguna pregunta, finaliza la conversación sin hacer otra pregunta.

No me siento bien. ¿y tú?

In [ ]:
#input_text = 'No mi siento bien. ¿y tú?'
#input_text = 'Bien, gracias. ¿Y te ti?'
input_text = 'un cafes'

In [ ]:
prompt = f"""
### Instrucción:
{input_text}

### Respuesta:
"""

result = pipeline(
    prompt,
    #max_length=128,
    max_new_tokens=64,
    do_sample=True,
    temperature = 0.1,
    top_p=0.75,
    top_k=50,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)
print(result[0]['generated_text'])

In [ ]:
prompt = """
A continuación se muestra una instrucción con errores gramaticales que describe una tarea. Escriba una respuesta que comience con "Si," con la corrección de la frase de instrucción. Cuando termine, complete correctamente la solicitud y luego detenga la conversación.
### Instrucción:
No mi siento bien. ¿y tú?

### Respuesta:
"""

result = pipeline(
    prompt,
    #max_length=128,
    max_new_tokens=64,
    do_sample=True,
    temperature = 0.1,
    top_p=0.75,
    top_k=50,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)
print(result[0]['generated_text'])

        "Instrucción": instruction,
        "Entrada": input_data,
        "Contexto": context,

In [ ]:
prompt = """
Aquí tienes una instrucción con errores gramaticales que describe una tarea. Repite la instrucción en un formato gramatical correcto. Completa adecuadamente la solicitud.
### Instrucción:
No mi siento bien. ¿y tú?

### Respuesta:
"""

result = pipeline(
    prompt,
    max_new_tokens=64,
    do_sample=True,
    temperature = 0.1,
    top_p=0.75,
    top_k=50,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)
print(result[0]['generated_text'])

In [ ]:
prompt = """
Aquí tienes una instrucción con errores gramaticales que describe una tarea. Repite la instrucción en un formato gramatical correcto. Completa adecuadamente la solicitud.
### Instrucción:
Bien, gracias. ¿Y a ti?

### Respuesta:
Ya veo, te refieres a Bien, gracias. ¿Y a ti? Muy bien.

### Instrucción:
Bien, gracias. ¿Y en ti?

### Respuesta:
Ya veo, te refieres a Bien, gracias. ¿Y a ti? Estoy genial.

### Instrucción:
Bien, gracias. ¿Y ti?

### Respuesta:
Ya veo, te refieres a Bien, gracias. ¿Y a ti? Me encuentro muy bien.

### Instrucción:
No mi siento bien. ¿y tú?

### Respuesta:
"""

result = pipeline(
    prompt,
    max_new_tokens=64,
    do_sample=True,
    temperature = 0.1,
    top_p=0.75,
    top_k=50,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)
print(result[0]['generated_text'])

### Test RAG